---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [5]:
def answer_one():
    count = spam_data[spam_data['target'] == 1]
    
    return (len(count) / len(spam_data)) * 100 #Your answer here
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vocabulary = list(CountVectorizer().fit(X_train).vocabulary_)  # list of vocabulary (convert dict tolist)
    vocabulary.sort(key=len, reverse=True)
    return vocabulary[0] #Your answer here
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
# from sklearn.feature_extraction.text import CountVectorizer

def answer_three():
    vectorizer = CountVectorizer()
    vectorized_X_train = vectorizer.fit_transform(X_train)
    
    classifier_multiNB = MultinomialNB(alpha=0.1)
    classifier_multiNB.fit(vectorized_X_train, y_train)
    predictions = classifier_multiNB.predict(vectorizer.transform(X_test))
    
    score = roc_auc_score(y_test,predictions)
    
    return score #Your answer here
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [20]:
def answer_four():
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    tfidf = TfidfVectorizer().fit(X_train)    
    X_train_tf = tfidf.transform(X_train)
    
    dataframe = pd.DataFrame(X_train_tf.toarray(), columns=tfidf.get_feature_names())
    liste = [(x,max(dataframe[x])) for x in dataframe.columns]
    small_tfidf = sorted(liste,key=lambda x:(x[1],x[0]))[:20]
    large_tfidf = sorted(liste,key=lambda x:(-x[1],x[0]))[:20]
    
    a = pd.Series([x[1] for x in small_tfidf], index=[x[0] for x in small_tfidf])
    b = pd.Series([x[1] for x in large_tfidf], index=[x[0] for x in large_tfidf])

    
    return (a, b)
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [81]:
def answer_five():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    
    vectorizer = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_tf = vectorizer.transform(X_train)
    
    model_NB = MultinomialNB(alpha=0.1)
    model_NB.fit(X_train_tf, y_train)
    
    predictions = model_NB.predict(vectorizer.transform(X_test))
    
    score = roc_auc_score(y_test,predictions)
    
    return score #Your answer here
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [116]:
def answer_six():
    from statistics import mean
    list1 = [len(text) for text in spam_data[spam_data['target'] == 1]['text']]
    list2 = [len(text) for text in spam_data[spam_data['target'] == 0]['text']]
    
    return (mean(list2),mean(list1)) #Your answer here
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [14]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [168]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
def answer_seven():
    
    X_train_documents_len = [len(text) for text in X_train]
    X_test_documents_len = [len(text) for text in X_test]
    
    vectorizer = TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_tf = vectorizer.transform(X_train)
    X_test_tf = vectorizer.transform(X_test)
    
    
    
    X_train_tf=  add_feature(X_train_tf, X_train_documents_len)
    X_test_tf = add_feature(X_test_tf, X_test_documents_len)
    
    
    model_svc = SVC(C=10000)
    model_svc.fit(X_train_tf,y_train)
    
    predictions = model_svc.predict(X_test_tf)
    
    score = roc_auc_score(y_test,predictions)
    
    return score #Your answer here
answer_seven()

0.95813668234215565

In [ ]:
def count_number(text):
    count = 0
    for i in text:
        if i.isdigit():
            count+=1
    return count

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [200]:
def answer_eight():
    from statistics import mean
    
    spam = [sum(digit.isnumeric() for digit in doc) for doc in spam_data[spam_data['target'] == 1]['text']]
    nospam = [sum(digit.isnumeric() for digit in doc) for doc in spam_data[spam_data['target'] == 0]['text']]
    
    return (mean(nospam),mean(spam))
answer_eight()

(0.2992746113989637, 15.76037483266399)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_nine():
    
    X_train_documents_len = [len(text) for text in X_train]
    X_test_documents_len = [len(text) for text in X_test]
    
    X_train_documents_digits = [sum(digit.isnumeric() for digit in doc) for doc in X_train]
    X_test_documents_digits = [sum(digit.isnumeric() for digit in doc) for doc in X_test]
    
    
    vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,3)).fit(X_train)
    X_train_tf = vectorizer.transform(X_train)
    X_test_tf = vectorizer.transform(X_test)
    
    X_train_tf=  add_feature(X_train_tf, X_train_documents_len)
    X_train_tf=  add_feature(X_train_tf, X_train_documents_digits)
    
    X_test_tf = add_feature(X_test_tf, X_test_documents_len)
    X_test_tf = add_feature(X_test_tf, X_test_documents_digits)
    
    model_svc = LogisticRegression(C=100)
    model_svc.fit(X_train_tf,y_train)
    
    predictions = model_svc.predict(X_test_tf)
    
    score = roc_auc_score(y_test,predictions)
    
    return score #Your answer here
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [6]:
def search_nonword(text):
    import re
    return re.findall('\W',text)

In [16]:
def answer_ten():
    from statistics import mean
    
    spam_nonWords = [len(search_nonword(doc)) for doc in spam_data[spam_data['target'] == 1]['text']]
    nospam_nonWords = [len(search_nonword(doc)) for doc in spam_data[spam_data['target'] == 0]['text']]
    
#     return (np.mean(spam_data.loc[spam_data['target']==0,'text'].str.count('\W')), 
#             np.mean(spam_data.loc[spam_data['target']==1,'text'].str.count('\W')))

#     return(mean(nospam_nonWords),mean(spam_nonWords))

    return (np.mean(spam_data[spam_data['target'] == 0]['text'].str.count('\W')), 
            np.mean(spam_data[spam_data['target'] == 1]['text'].str.count('\W')))
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
def answer_eleven():
    
    X_train_documents_len = [len(text) for text in X_train]
    X_test_documents_len = [len(text) for text in X_test]
    
    X_train_documents_digits = [sum(digit.isnumeric() for digit in doc) for doc in X_train]
    X_test_documents_digits = [sum(digit.isnumeric() for digit in doc) for doc in X_test]
    
    X_train_nonWords = [len(search_nonword(doc)) for doc in X_train]
    X_test_nonWords = [len(search_nonword(doc)) for doc in X_test]

    
    vectorizer = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb')
    
    X_train_tf = vectorizer.fit_transform(X_train)
    X_test_tf = vectorizer.transform(X_test)
    
    X_train_tf=  add_feature(X_train_tf, X_train_documents_len)
    X_train_tf=  add_feature(X_train_tf, X_train_documents_digits)
    X_train_tf = add_feature(X_train_tf, X_train_nonWords)
    
    X_test_tf = add_feature(X_test_tf, X_test_documents_len)
    X_test_tf = add_feature(X_test_tf, X_test_documents_digits)
    X_test_tf = add_feature(X_test_tf, X_test_nonWords)
    
    
    
    model_LR = LogisticRegression(C=100)
    model_LR.fit(X_train_tf,y_train)
    
    predictions = model_LR.predict(X_test_tf)
    
    score = roc_auc_score(y_test,predictions)
    
    feature_names = np.array(vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model_LR.coef_[0].argsort()
    small_coeffs = list(feature_names[sorted_coef_index[:10]])
    large_coeffs = list(feature_names[sorted_coef_index[:-11:-1]])
    
    return (score, small_coeffs, large_coeffs)
    
    
    
    return score
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])